## AutoEncoder Model - Fraud

In [0]:
%config IPCompleter.greedy=True
#%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
#%load_ext autotime

In [2]:
import pandas as pd
import numpy as np

from keras.models import Model
from keras.layers import Input,Dense

Using TensorFlow backend.


In [3]:
df = pd.read_csv('PCA_standardized.csv')

FileNotFoundError: ignored

In [0]:
df.head()

In [0]:
df = df.iloc[:,1:]

In [0]:
df.head()

In [0]:
df.describe()

In [0]:
train_df = df.values

In [0]:
input_size = 8
hidden_size = 4
code_size = 2

#autoencoder
input_layer = Input(shape=(input_size,)) 
hidden_1 = Dense(hidden_size, activation='relu')(input_layer)
code = Dense(code_size,activation='relu')(hidden_1)
hidden_2 = Dense(hidden_size, activation='relu')(code)
output_layer = Dense(input_size, activation='tanh')(hidden_2)

autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [0]:
output=autoencoder.fit(train_df,train_df, epochs = 10)

In [0]:
ans = autoencoder.predict(train_df)

In [0]:
ans_df = pd.DataFrame(ans)

In [0]:
ans_df.head()

In [0]:
ans_df.to_csv('ans_Reconstructed.csv',index=False)

In [0]:
ans_df.iloc[:,2].mean()

In [0]:
pd.DataFrame(train_df).iloc[:,2].mean()

In [0]:
#n=1
manhattan = (ans_df - pd.DataFrame(train_df)).abs()
manhattan_score = manhattan.sum(axis = 1)
manhattan_score

In [0]:
#n=2
euclidean = (((ans - pd.DataFrame(train_df)).abs())**2).sum(axis=1)
euclidean_score = euclidean**(1/2)
euclidean_score

In [0]:
manhattan_score=pd.DataFrame(manhattan_score)
euclidean_score=pd.DataFrame(euclidean_score)
fraud_score_AE= pd.concat([manhattan_score,euclidean_score], axis=1)
fraud_score_AE.columns=['Manhattan Score','Euclidean Score']

In [0]:
fraud_score_AE.insert(0,'Record',fraud_score_AE.index+1)

In [0]:
fraud_score_AE.describe()

In [0]:
fraud_score_AE.to_csv('fraud_score_AE.csv',index=False)

In [0]:
fraud_score_AE.head()

In [0]:
fraud_score_AE['Euclidean Rank'] = fraud_score_AE['Euclidean Score'].rank(ascending=1)
fraud_score_AE['Manhattan Rank'] = fraud_score_AE['Manhattan Score'].rank(ascending=1)
fraud_score_AE.head()

In [0]:
fraud_score_AE.to_csv('fraud_ScoreAndRanks_AE.csv',index=False)

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.distplot(fraud_score_AE['Manhattan Score'],bins=100, kde=True)
#plt.title('Distribution of :'+col1)
#plt.ylabel('Density of probability', fontsize=12)
#plt.xlabel(col1, fontsize=12)
#plt.savefig("imgs/"+col1+".png")
plt.show()

In [0]:
colName='Manhattan Score'
xhigh =1.25
#sns.plt.xlim(0,xhigh)
temp = fraud_score_AE[fraud_score_AE[colName] <= xhigh]
sns.distplot(temp[colName],bins=10000, kde=True)
plt.title('Distribution of :'+colName)
#plt.xscale('log')
plt.ylabel('Density of probability', fontsize=12)
plt.ylim((0.00, 20)) 
#plt.xlim((0.00, 25)) 
plt.xlabel(colName, fontsize=12)
#plt.savefig("imgs/"+colName+".png")
plt.show()

In [0]:
colName='Euclidean Score'
xhigh =1.25
#sns.plt.xlim(0,xhigh)
#temp = fraud_score_AE
temp = fraud_score_AE[fraud_score_AE[colName] <= xhigh]
sns.distplot(temp[colName],bins=10000, kde=True)
plt.title('Distribution of :'+colName)
#plt.xscale('log')
plt.ylabel('Density of probability', fontsize=12)
plt.ylim((0.00, 25)) 
#plt.xlim((0.00, 25)) 
plt.xlabel(colName, fontsize=12)
#plt.savefig("imgs/"+colName+".png")
plt.show()

In [0]:
bins=10
colName='Euclidean Score'
new_data = pd.qcut(fraud_score_AE[colName], bins, labels=list(range(bins)))

In [0]:
type(new_data)
new_data.value_counts()

In [0]:
pd.DataFrame(new_data).head()